# Bay Area Water Quality Data Analysis
### ENERES 131: Final Project

#### Team Members: Damaris Arteaga,  Nidhi Nayak, Vishali Kirthi Vallioor, Timothy Zheng

![Water Quality](https://plus.unsplash.com/premium_photo-1710628263718-367b1cf5828f?q=80&w=2940&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D)

---

## Table of Contents and Deliverables
1. <a href="#abstract"><span style="color:red;">Section 1:</span> Project Contributions</a>
2. <a href="#abstract"><span style="color:red;">Section 2:</span> Abstract</a>
3. <a href="#project-background"><span style="color:red;">Section 3:</span> Project Background</a>
4. <a href="#project-objective"><span style="color:red;">Section 4:</span> Project Objective</a>
5. <a href="#input-data-description"><span style="color:red;">Section 5:</span> Input Data Description</a>
6. <a href="#data-cleaning"><span style="color:red;">Section 6:</span> Data Cleaning</a>
7. <a href="#eda"><span style="color:red;">Section 7:</span> Data Summary and Exploratory Data Analysis</a>
8. <a href="#forecasting-modeling"><span style="color:red;">Section 8:</span> Forecasting and Prediction Modeling </a>
   - <a href="#core-prediction"><span style="color:red;">Subsection 8.1:</span> Core Prediction Question</a>
   - <a href="#supporting-prediction"><span style="color:red;">Subsection 8.2:</span> Supporting Question</a>
   - <a href="#temporal-prediction"><span style="color:red;">Subsection 8.3:</span> Temporal Prediction Question</a>
9. <a href="#conclusions"><span style="color:red;">Section 9:</span> Interpretation and Conclusions</a>
10. <a href="#references"><span style="color:red;">Section 10:</span> Words Cited</a>

---

## <a id="abstract"></a>Section 1: Project Contributions

- #### Damaris Arteaga: ####
  **Contributions:**

  **Hours Contributed:**
  
- #### Nidhi Nayak: ####
  **Contributions:**

  **Hours Contributed:**
  
- #### Vishali Vallioor: ####
  **Contributions:**

  **Hours Contributed:**
  
- #### Timothy Zheng: ####
  **Contributions:**

  **Hours Contributed:**
  

---

## <a id="abstract"></a>Section 2: Abstract

---

## <a id="project-background"></a>Section 3: Project Background

The Bay Area, one of the most densely populated urban areas in California, is home to a variety of water bodies, including rivers, lakes, and coastal beaches, all of which play a critical role in supporting both human and wildlife populations. However, due to increasing urbanization, industrial activities, and climate changes, maintaining water quality here has proven to be a substantial challenge. Among the various water quality parameters, turbidity levels, Enterococcus levels, and dissolved oxygen (DO) concentrations are particularly important indicators of aquatic health and public safety.

**Turbidity** refers to the cloudiness or haziness of water caused by the presence of suspended particles such as soil, algae, and organic matter. High turbidity can block sunlight, directly impacting aquatic plant life. It can also carry harmful pathogens and chemicals, impacting both the ecosystem and human health. In the Bay Area, turbidity levels fluctuate based on weather conditions, such as rainfall and stormwater runoff, which can cause large amounts of sediment to enter water bodies.  

**Enterococcus** is a type of bacteria commonly found in human and animal waste. Elevated levels of Enterococcus in recreational waters are an indicator of possible fecal contamination. The presence of these bacteria signifies a high health risk, especially in recreational areas where people perform water-based activities like swimming or surfing. High concentrations of Enterococcus are correlated to numerous gastrointestinal illnesses, making it a key factor for beach monitoring. By assessing these levels in an accurate and timely manner, public health agencies can issue warnings and implement adequate safety measures.

**Dissolved Oxygen (DO)** measures the amount of oxygen available for aquatic organisms, including fish, invertebrates, and plants. Adequate DO levels are essential for the survival of these organisms. However, fluctuations in DO can indicate problems such as excessive nutrient loading, algal blooms, or organic matter decomposition. Low DO levels, often referred to as hypoxia, can lead to "dead zones," areas where aquatic life cannot thrive. In the Bay Area, DO levels are influenced by various factors like water temperature, salinity, and pollution. Monitoring DO levels is extremely important to maintain a balanced and thriving ecosystem and wildlife. 

Given the complex and interconnected nature of these three water quality parameters, it is of utmost importance to quantitatively analyze their fluctuations over time. Basic water quality monitoring revolves around periodic sampling, which, while providing valuable data, is unable to capture the trends and variability of these parameters. By performing advanced data analysis, we can predict these feature levels with greater accuracy and speed. This can aid policymakers, environmental agencies, and public health officials to make more informed decisions, issue timely warnings, and implement more effective water management practices.


---

## <a id="project-objective"></a>Section 4: Project Objective


---

## <a id="input-data-description"></a>Section 5: Input Data Description

---

## <a id="data-cleaning"></a>Section 6: Data Cleaning

---

## <a id="eda"></a>Section 7: Data Summary and Exploratory Data Analysis

---

## <a id="forecasting-modeling"></a>Section 8: Forecasting and Prediction Modeling

---

### <a id="core-prediction"></a>Subsection 8.1: Core Prediction Question

---

### <a id="supporting-prediction"></a>Subsection 8.2: Supporting Question

---

### <a id="temporal-prediction"></a>Subsection 8.3: Temporal Prediction Question

---

## <a id="conclusions"></a>Section 9: Interpretation and Conclusions

---

## <a id="references"></a>Section 10: Works Cited

---